In [1]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

### Data loading

In [2]:
train_dataset = "C:/Users/pickme1124/Downloads/data/train"
test_dataset = "C:/Users/pickme1124/Downloads/data/test"

image_size = 256
classes = 12

In [3]:
#Build train generator

image_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0
)

train_generator = image_datagenerator.flow_from_directory(train_dataset,
                                                    target_size=(image_size, image_size),
                                                    subset='training',
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    batch_size = 4750,
                                                    seed= 9)
X,Y = train_generator.next()

Found 4750 images belonging to 12 classes.


In [4]:
# Create test data set
class_mapping = { train_generator.class_indices[k]:k for k in train_generator.class_indices}

def create_test_dataset(test_images_dir, image_size, rescale = 1):
    filenames = []
    test_data = []
    
    for img_file in os.listdir(test_images_dir):
        filenames.append(img_file)

        img = tf.keras.preprocessing.image.load_img(test_images_dir + '/' + img_file)
        img_new = img.resize([image_size, image_size])
        img_array = tf.keras.preprocessing.image.img_to_array(img_new)
        if rescale is not 1:
            img_array = img_array * 1./255
        test_data.append(img_array)
    
    test_data = np.array(test_data,dtype=np.float32)
    
    return test_data, filenames 

test_data, filenames  = create_test_dataset(test_dataset,image_size, rescale = 1./255)

In [5]:
#Prepare submisiion file
def prepare_submission(submission_file, model):
    predicted_class = []
    for img in test_data:
        img_instance = np.expand_dims(img, axis=0)
        img_instance_class = model.predict(img_instance)
        img_instance_class = np.argmax(img_instance_class,axis=1)
        img_instance_class = class_mapping.get(img_instance_class[0], np.nan)
        predicted_class.append(img_instance_class)
        pass

    results = pd.DataFrame({"file":filenames,"species":predicted_class})
    results.to_csv(submission_file, index = False)

### Model building and evaluation

In [9]:
#Cross validation
num_folds = 5

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

kfold = KFold(n_splits=num_folds, random_state=1, shuffle=True)

# Merge inputs and targets
inputs = np.concatenate((x_train, x_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [10]:
#Train the model
def train_model(no_epochs,batch_size, model):
    
    # Define per-fold score containers
    acc_score_per_fold = []

    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(inputs, targets):

        # Fit data to model
        history = model.fit(inputs[train], targets[train],
                      batch_size=batch_size,
                      epochs=no_epochs,
                      verbose=2)

        # Generate generalization metrics
        loss, accuracy = model.evaluate(inputs[test], targets[test], verbose=0)
        acc_score_per_fold.append(accuracy)

        print(f'> Fold {fold_no} Accuracy score: {accuracy}')
        # Increase fold number
        fold_no = fold_no + 1

    # == Provide average scores ==
    print('Average accuracy score for all folds:')
    print(f'> Accuracy score: {np.mean(acc_score_per_fold)} (+- {np.std(acc_score_per_fold)})')
    print('------------------------------------------------------------------------')

### Model V1

In [8]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()
#normalize data
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3),strides = (2,2), activation='relu'))
#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(image_size, activation='relu'))
#Add Output Layer
model.add(tf.keras.layers.Dense(classes, activation='softmax'))

#Model Summary
model.summary()

#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 127, 127, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 254016)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               65028352  
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
Total params: 65,033,228
Trainable params: 65,033,228
Non-trainable params: 0
_________________________________________________________________


In [9]:
train_model(10, 100, model)

Epoch 1/10
38/38 - 44s - loss: 3.9157 - accuracy: 0.1761
Epoch 2/10
38/38 - 39s - loss: 2.0369 - accuracy: 0.3203
Epoch 3/10
38/38 - 37s - loss: 1.6504 - accuracy: 0.4505
Epoch 4/10
38/38 - 36s - loss: 1.3688 - accuracy: 0.5524
Epoch 5/10
38/38 - 39s - loss: 1.1821 - accuracy: 0.6266
Epoch 6/10
38/38 - 38s - loss: 0.9520 - accuracy: 0.7058
Epoch 7/10
38/38 - 38s - loss: 0.7536 - accuracy: 0.7882
Epoch 8/10
38/38 - 37s - loss: 0.6140 - accuracy: 0.8392
Epoch 9/10
38/38 - 37s - loss: 0.4571 - accuracy: 0.8974
Epoch 10/10
38/38 - 37s - loss: 0.3485 - accuracy: 0.9287
> Fold 1 Accuracy score: 0.49473685026168823
Epoch 1/10
38/38 - 37s - loss: 0.6349 - accuracy: 0.8276
Epoch 2/10
38/38 - 37s - loss: 0.4531 - accuracy: 0.8832
Epoch 3/10
38/38 - 37s - loss: 0.3087 - accuracy: 0.9355
Epoch 4/10
38/38 - 37s - loss: 0.2064 - accuracy: 0.9663
Epoch 5/10
38/38 - 37s - loss: 0.1510 - accuracy: 0.9808
Epoch 6/10
38/38 - 40s - loss: 0.1085 - accuracy: 0.9916
Epoch 7/10
38/38 - 37s - loss: 0.0966 - ac

In [10]:
prepare_submission("submissions/submission_v1.csv", model)

### Model V2

In [8]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()
#normalize data
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3),strides = (2,2), activation='relu'))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))
#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))
#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (3,3)))
#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(image_size, activation='relu'))
#Add Output Layer
model.add(tf.keras.layers.Dense(classes, activation='softmax'))

#Model Summary
model.summary()

#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 127, 127, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               1

In [9]:
train_model(10, 100, model)

Epoch 1/10
38/38 - 211s - loss: 2.2022 - accuracy: 0.2276
Epoch 2/10
38/38 - 197s - loss: 1.6599 - accuracy: 0.4111
Epoch 3/10
38/38 - 194s - loss: 1.3735 - accuracy: 0.5211
Epoch 4/10
38/38 - 191s - loss: 1.2101 - accuracy: 0.5805
Epoch 5/10
38/38 - 197s - loss: 1.0834 - accuracy: 0.6232
Epoch 6/10
38/38 - 189s - loss: 0.9121 - accuracy: 0.6913
Epoch 7/10
38/38 - 198s - loss: 0.7799 - accuracy: 0.7403
Epoch 8/10
38/38 - 188s - loss: 0.5928 - accuracy: 0.8079
Epoch 9/10
38/38 - 173s - loss: 0.4469 - accuracy: 0.8503
Epoch 10/10
38/38 - 174s - loss: 0.3227 - accuracy: 0.9032
> Fold 1 Accuracy score: 0.5589473843574524
Epoch 1/10
38/38 - 183s - loss: 0.6419 - accuracy: 0.8108
Epoch 2/10
38/38 - 174s - loss: 0.3726 - accuracy: 0.8953
Epoch 3/10
38/38 - 176s - loss: 0.2591 - accuracy: 0.9287
Epoch 4/10
38/38 - 173s - loss: 0.1659 - accuracy: 0.9553
Epoch 5/10
38/38 - 176s - loss: 0.0809 - accuracy: 0.9834
Epoch 6/10
38/38 - 173s - loss: 0.0435 - accuracy: 0.9926
Epoch 7/10
38/38 - 177s - l

In [11]:
prepare_submission("submissions/submission_v2.csv", model)

### Model V3

In [6]:
#Clear any previous model from memory
tf.keras.backend.clear_session()

#Initialize model
model = tf.keras.models.Sequential()
#normalize data
model.add(tf.keras.layers.InputLayer(input_shape=(image_size,image_size,3,)))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3),strides = (2,2), activation='relu'))
# Normalization layer
model.add(tf.keras.layers.BatchNormalization())
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(64, kernel_size=(3,3), strides = (1,1), activation='relu'))
#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (2,2)))
#Add Conv Layer
model.add(tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides = (1,1), activation='relu'))
#Add Max Pool layer
model.add(tf.keras.layers.MaxPool2D(pool_size = (3,3)))
#Add Dense Layers after flattening the data
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(image_size, activation='relu'))
# Dropout
# is used to nullify the outputs that are very close to zero and thus can cause overfitting.
model.add(tf.keras.layers.Dropout(0.2))
#Add Output Layer
model.add(tf.keras.layers.Dense(classes, activation='softmax'))

#Model Summary
model.summary()

#Specify Loass and Optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 127, 127, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 127, 127, 64)      256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0

In [11]:
train_model(10, 100, model)

Epoch 1/10
38/38 - 242s - loss: 3.8269 - accuracy: 0.1671
Epoch 2/10
38/38 - 230s - loss: 1.8447 - accuracy: 0.3842
Epoch 3/10
38/38 - 233s - loss: 1.3857 - accuracy: 0.5171
Epoch 4/10
38/38 - 228s - loss: 1.0379 - accuracy: 0.6368
Epoch 5/10
38/38 - 230s - loss: 0.7811 - accuracy: 0.7384
Epoch 6/10
38/38 - 232s - loss: 0.5369 - accuracy: 0.8155
Epoch 7/10
38/38 - 231s - loss: 0.4068 - accuracy: 0.8674
Epoch 8/10
38/38 - 235s - loss: 0.2969 - accuracy: 0.8995
Epoch 9/10
38/38 - 229s - loss: 0.2009 - accuracy: 0.9313
Epoch 10/10
38/38 - 234s - loss: 0.1444 - accuracy: 0.9574
> Fold 1 Accuracy score: 0.40526315569877625
Epoch 1/10
38/38 - 236s - loss: 0.5723 - accuracy: 0.8471
Epoch 2/10
38/38 - 231s - loss: 0.2848 - accuracy: 0.9161
Epoch 3/10
38/38 - 233s - loss: 0.1756 - accuracy: 0.9434
Epoch 4/10
38/38 - 232s - loss: 0.1137 - accuracy: 0.9639
Epoch 5/10
38/38 - 230s - loss: 0.1035 - accuracy: 0.9642
Epoch 6/10
38/38 - 231s - loss: 0.0768 - accuracy: 0.9755
Epoch 7/10
38/38 - 233s - 

In [12]:
prepare_submission("submissions/submission_v3.csv", model)